# Pandas ile Eksik Verilerle Başa Çıkma (Kapsamlı Rehber)

Bu rehber, veri analizinin en yaygın ve önemli sorunlarından biri olan **eksik verilerle (Missing Data)** nasıl başa çıkılacağını kapsamlı bir şekilde ele almaktadır. Eksik verilerin doğru bir şekilde yönetilmesi, analizlerinizin ve makine öğrenmesi modellerinizin güvenilirliği için hayati önem taşır.

## 1. Eksik Veri Türleri ve Neden Önemlidir?

Eksik veriler genellikle 3 kategoriye ayrılır:
1.  **Tamamen Rastlantısal Kayıp (MCAR - Missing Completely at Random):** Verinin eksik olmasının diğer verilerle hiçbir ilişkisi yoktur. (Örn: Bir anket sorusunun rastgele atlanması).
2.  **Rastlantısal Kayıp (MAR - Missing at Random):** Verinin eksik olması, veri setindeki diğer gözlemlenen değişkenlerle ilişkilidir. (Örn: Erkeklerin depresyon skorlarını kadınlara göre daha sık boş bırakması. Eksiklik 'cinsiyet' değişkenine bağlıdır).
3.  **Rastlantısal Olmayan Kayıp (MNAR - Missing Not at Random):** Verinin eksik olması, o değişkenin kendisiyle ilişkilidir. (Örn: Yüksek maaş alan kişilerin maaş bilgisini girmek istememesi. Eksiklik 'maaş' değişkeninin kendisine bağlıdır).

Bu türleri anlamak, doğru başa çıkma stratejisini seçmemize yardımcı olur.

## 2. Kurulum ve Örnek Veri Seti Oluşturma

Konuyu daha iyi anlamak için çeşitli eksik veri türleri içeren sentetik bir veri seti oluşturalım.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

data = {
    'yas': [25, 30, np.nan, 45, 50, 60, 22, np.nan, 35],
    'cinsiyet': ['Erkek', 'Kadın', 'Erkek', np.nan, 'Kadın', 'Erkek', 'Kadın', 'Erkek', 'Kadın'],
    'maas': [50000, 60000, 45000, 80000, np.nan, 120000, 30000, 40000, 70000],
    'departman': ['IT', 'İK', 'Pazarlama', 'IT', 'İK', np.nan, 'Pazarlama', 'IT', 'İK'],
    'deneyim_yili': [2, 7, 4, 20, 25, 35, 1, 3, 10],
    'son_performans_notu': [85, 90, 88, 95, np.nan, np.nan, 82, 84, 91]
}

df = pd.DataFrame(data)

print("Örnek Veri Seti:")
display(df)

## 3. Eksik Verileri Tespit Etme

İlk adım, veri setimizde nerede ve ne kadar eksik veri olduğunu anlamaktır.

In [ ]:
# Her sütundaki eksik değerlerin toplam sayısı
print("Sütunlardaki Eksik Veri Sayısı:")
print(df.isnull().sum())

# Her sütundaki eksik değerlerin yüzdesi
print("
Sütunlardaki Eksik Veri Yüzdesi (%):")
print(df.isnull().mean() * 100)

### Görselleştirme ile Tespit

`seaborn.heatmap` kullanarak eksik verilerin bir haritasını çıkarabiliriz. Bu, özellikle büyük veri setlerinde eksikliklerin dağılımını anlamak için çok etkilidir.

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis', yticklabels=False)
plt.title('Eksik Veri Haritası')
plt.show()

# Not: Daha gelişmiş görselleştirmeler için `missingno` kütüphanesi de harika bir seçenektir. (pip install missingno)

## 4. Strateji 1: Eksik Verileri Silme (Deletion)

En basit yöntemdir ancak veri kaybına neden olduğu için dikkatli kullanılmalıdır.

In [ ]:
# Herhangi bir eksik değer içeren satırları silme
df_dropped_rows = df.dropna()
print("Herhangi bir NaN içeren satırlar silindikten sonraki veri:")
display(df_dropped_rows)

# Belirli bir sütundaki eksik değere göre satır silme
df_dropped_subset = df.dropna(subset=['maas', 'cinsiyet'])
print("
'maas' veya 'cinsiyet' sütununda NaN olan satırlar silindikten sonra:")
display(df_dropped_subset)

## 5. Strateji 2: Eksik Verileri Doldurma (Imputation)

Daha sık tercih edilen yöntem, eksik verileri belirli bir stratejiye göre doldurmaktır.

### a. Basit Doldurma Yöntemleri

In [ ]:
# Sabit bir değerle doldurma
df_filled_constant = df.copy()
df_filled_constant['departman'].fillna('Bilinmiyor', inplace=True)
print("'departman' sabit değerle dolduruldu:")
display(df_filled_constant.head())

# Önceki değerle doldurma (Forward Fill) - Zaman serileri için kullanışlıdır
df_ffill = df.copy()
df_ffill['son_performans_notu'].fillna(method='ffill', inplace=True)
print("\n'son_performans_notu' ffill ile dolduruldu:")
display(df_ffill)

# Sonraki değerle doldurma (Backward Fill)
df_bfill = df.copy()
df_bfill['son_performans_notu'].fillna(method='bfill', inplace=True)
print("\n'son_performans_notu' bfill ile dolduruldu:")
display(df_bfill)

### b. İstatistiksel Doldurma Yöntemleri

In [ ]:
df_imputed = df.copy()

# Sayısal sütunları ortalama veya medyan ile doldurma
# Medyan, aykırı değerlere karşı daha dayanıklıdır.
yas_medyan = df_imputed['yas'].median()
df_imputed['yas'].fillna(yas_medyan, inplace=True)

maas_ortalama = df_imputed['maas'].mean()
df_imputed['maas'].fillna(maas_ortalama, inplace=True)

# Kategorik sütunları en sık tekrar eden değer (mod) ile doldurma
cinsiyet_mod = df_imputed['cinsiyet'].mode()[0]
df_imputed['cinsiyet'].fillna(cinsiyet_mod, inplace=True)

print('İstatistiksel doldurma sonrası veri:')
display(df_imputed)

### c. Gelişmiş Doldurma: Gruplara Özel Doldurma

Bu çok daha güçlü bir yöntemdir. Eksik bir değeri, ait olduğu grubun istatistiklerine göre doldururuz. Örneğin, eksik maaş bilgisini, kişinin bulunduğu **departmanın ortalama maaşına** göre doldurabiliriz.

In [ ]:
df_group_imputed = df.copy()

# Departmanlara göre ortalama maaşı hesapla ve eksik maaşları doldur
df_group_imputed['maas'] = df_group_imputed.groupby('departman')['maas'].transform(
    lambda x: x.fillna(x.mean())
)

print('Departman ortalamasına göre doldurulmuş maaşlar:')
display(df_group_imputed)

### d. İnterpolasyon ile Doldurma

`interpolate()` metodu, özellikle sıralı veya zaman serisi verilerinde eksik değerleri, komşu değerlere dayalı olarak tahmin eder.

In [ ]:
df_interpolated = df.copy()

# Lineer interpolasyon uygula
df_interpolated['son_performans_notu'].interpolate(method='linear', inplace=True)

print('İnterpolasyon ile doldurulmuş performans notları:')
display(df_interpolated)

## 6. Strateji 3: Gelişmiş Doldurma (Scikit-learn)

Daha karmaşık senaryolar için `scikit-learn` kütüphanesi, diğer özelliklerin ilişkilerini de dikkate alarak doldurma yapabilen güçlü araçlar sunar.

- **`KNNImputer`**: Eksik bir değeri, ona en yakın 'k' komşusunun değerlerinin ortalamasıyla doldurur.
- **`IterativeImputer`**: Her bir eksik özellik için bir regresyon modeli oluşturur ve diğer özellikleri kullanarak eksik değerleri tahmin eder. Genellikle en güçlü yöntemlerden biridir.

Bu yöntemler, `Scikit-Learn_Rehberi.ipynb` dosyasında daha detaylı incelenebilir.

In [ ]:
from sklearn.impute import KNNImputer

df_knn = df[['yas', 'maas', 'deneyim_yili', 'son_performans_notu']].copy()

imputer = KNNImputer(n_neighbors=2)
df_knn_imputed = imputer.fit_transform(df_knn)

# Sonucu DataFrame'e çevirelim
df_knn_imputed = pd.DataFrame(df_knn_imputed, columns=df_knn.columns)

print('KNNImputer ile Doldurulmuş Veri:')
display(df_knn_imputed)

## Sonuç ve Hangi Yöntemi Seçmeli?

En iyi yöntemi seçmek, probleme ve veriye bağlıdır:

- **Hızlı ve basit bir çözüm için:** `mean`/`median`/`mode` ile doldurma genellikle yeterlidir.
- **Veri kaybını göze alabiliyorsanız ve eksiklik azsa:** `dropna()` kullanılabilir.
- **Zaman serisi veya sıralı verilerle çalışıyorsanız:** `ffill`, `bfill` veya `interpolate` iyi seçeneklerdir.
- **Değişkenler arasında ilişki olduğunu düşünüyorsanız ve daha doğru tahminler istiyorsanız:** Gruplara özel doldurma veya `scikit-learn`'ün `KNNImputer` / `IterativeImputer`'ı gibi gelişmiş yöntemler en iyi sonuçları verecektir.